In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
cath_df = pd.read_csv("./data/cath_moments.tsv", sep='\t', header=None) # For Train
ecod_df = pd.read_csv("./data/ecod_moments.tsv", sep='\t', header=None) # For eval

In [ ]:
cath_info = cath_df.info()

In [ ]:
ecod_info = ecod_df.info()

In [ ]:
# Summarize shapes, classes, and a preview
cath_summary = {
    "shape": cath_df.shape,
    "unique_classes": cath_df[0].nunique(),
    "class_distribution": cath_df[0].value_counts(),
    "head": cath_df.head()
}

ecod_summary = {
    "shape": ecod_df.shape,
    "unique_classes": ecod_df[0].nunique(),
    "class_distribution": ecod_df[0].value_counts(),
    "head": ecod_df.head()
}

In [ ]:
print(cath_summary)

In [ ]:
print(ecod_summary)

In [ ]:
cath_df.iloc[0]

In [ ]:
for i in range(0,2):
    print(type(cath_df.iloc[0,i]))

In [ ]:
# Plot class distribution for CATH
cath_class_counts = cath_df[0].value_counts()
plt.figure(figsize=(12, 4))
plt.hist(cath_class_counts, bins=30, edgecolor='black')
plt.title('Class Frequency Distribution in CATH Dataset')
plt.xlabel('Number of proteins per class')
plt.ylabel('Number of classes')
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot class distribution for ECOD
ecod_class_counts = ecod_df[0].value_counts()
plt.figure(figsize=(12, 4))
plt.hist(ecod_class_counts, bins=20, edgecolor='black')
plt.title('Class Frequency Distribution in ECOD Dataset')
plt.xlabel('Number of proteins per class')
plt.ylabel('Number of classes')
plt.grid(True)
plt.tight_layout()
plt.show()


### Visualizing Class Imbalance

Each protein in the dataset is associated with a structural class label (column 0), which identifies its 3D(1D in this case) shape category. Understanding how many proteins fall into each class is critical because the dataset is not balanced—some classes have many proteins, while others have very few.

To quantify this, we use `value_counts()` to compute the frequency of each class label and visualize the distribution with a histogram.

#### Why this is important:
During training, we will create pairs of proteins to determine structural similarity. If a particular class contains many proteins, it will generate significantly more pairs, which may bias the model toward frequently occurring classes. This can lead to overfitting and poor generalization, especially on underrepresented classes.

By plotting the class frequency distribution:
- We confirm whether class imbalance is present.
- We motivate the need for sampling strategies such as `WeightedRandomSampler` to correct for this imbalance during training.

This analysis is a key step in understanding the structure of the data and informing how we design the training process.


In [2]:
import sys
import numpy as np

print("Python version:", sys.version)
print("NumPy version:", np.__version__)


Python version: 3.12.2 | packaged by Anaconda, Inc. | (main, Feb 27 2024, 12:57:28) [Clang 14.0.6 ]
NumPy version: 1.26.4


In [3]:
import pandas as pd
import torch
import importlib
import train
import dataset
import time
import os
importlib.reload(train)
importlib.reload(dataset)
import random

random.seed(42)


from train import train_model, test_model_on_ecod

[INFO] Using 12 CPU threads
[INFO] Using 12 CPU threads


In [4]:
# Load datasets
cath_df = pd.read_csv("./data/cath_moments.tsv", sep='\t', header=None).dropna(axis=1)
ecod_df = pd.read_csv("./data/ecod_moments.tsv", sep='\t', header=None).dropna(axis=1)

print(f"CATH: {cath_df.shape}, ECOD: {ecod_df.shape}")
features,labels=None,None


CATH: (2685, 3923), ECOD: (761, 3923)


In [7]:
num_proteins = 2685

In [8]:
print("Expected pairs:",num_proteins*(num_proteins-1)//2)

Expected pairs: 3603270


In [5]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
lr = 1e-3

model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=32, input_dim=input_dim,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)bb

Training on: cpu (CPU)
[TensorBoard] Logging to: tensorboard_logs/baseline_h32_bs128_lr0.001_ep30_20250401-013606
[INFO] Loading Dataloader
[INFO] Initializing Streaming Dataset from DataFrame of size 2685
[INFO] Streaming V2 init done in 0.63 seconds
[INFO] Creating traindata with 3242943
[INFO] Creating valdata with 360327
Training model (hidden_dim=32) for 30 epochs...


/opt/anaconda3/envs/tf_keras/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 1/30 | Train Loss: 3512.7006 | Val Loss: 248.7990 | ROC AUC: 0.985 | PR AUC: 0.612 | MCC: 0.287


Epoch 2/30:   0%|          | 0/25336 [00:00<?, ?it/s]

...

Epoch 28/30 | Train Loss: 1989.0283 | Val Loss: 121.1449 | ROC AUC: 0.984 | PR AUC: 0.627 | MCC: 0.376


Epoch 29/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 29/30 | Train Loss: 1993.8984 | Val Loss: 119.3949 | ROC AUC: 0.984 | PR AUC: 0.618 | MCC: 0.375


Epoch 30/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 30/30 | Train Loss: 1989.6741 | Val Loss: 154.8932 | ROC AUC: 0.985 | PR AUC: 0.623 | MCC: 0.347


In [8]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
lr = 1e-3

model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=64, input_dim=input_dim,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

Training on: cpu (CPU)
[TensorBoard] Logging to: tensorboard_logs/baseline_h64_bs128_lr0.001_ep30_20250401-044602
[INFO] Loading Dataloader
[INFO] Initializing Streaming Dataset from DataFrame of size 2685
[INFO] Streaming V2 init done in 0.90 seconds
[INFO] Creating traindata with 3242943
[INFO] Creating valdata with 360327
Training model (hidden_dim=64) for 30 epochs...


/opt/anaconda3/envs/tf_keras/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 1/30 | Train Loss: 2790.1215 | Val Loss: 133.2154 | ROC AUC: 0.986 | PR AUC: 0.627 | MCC: 0.382


Epoch 2/30:   0%|          | 0/25336 [00:00<?, ?it/s]

...

Epoch 28/30 | Train Loss: 1029.9672 | Val Loss: 96.3476 | ROC AUC: 0.986 | PR AUC: 0.643 | MCC: 0.426


Epoch 29/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 29/30 | Train Loss: 1026.2533 | Val Loss: 108.3883 | ROC AUC: 0.987 | PR AUC: 0.647 | MCC: 0.416


Epoch 30/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 30/30 | Train Loss: 1018.7080 | Val Loss: 74.7417 | ROC AUC: 0.984 | PR AUC: 0.638 | MCC: 0.458


In [9]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
lr = 1e-3

model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=128, input_dim=input_dim,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

Training on: cpu (CPU)
[TensorBoard] Logging to: tensorboard_logs/baseline_h128_bs128_lr0.001_ep30_20250401-052913
[INFO] Loading Dataloader
[INFO] Initializing Streaming Dataset from DataFrame of size 2685
[INFO] Streaming V2 init done in 0.92 seconds
[INFO] Creating traindata with 3242943
[INFO] Creating valdata with 360327
Training model (hidden_dim=128) for 30 epochs...


/opt/anaconda3/envs/tf_keras/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 1/30 | Train Loss: 2328.2184 | Val Loss: 141.1325 | ROC AUC: 0.986 | PR AUC: 0.628 | MCC: 0.363


Epoch 2/30:   0%|          | 0/25336 [00:00<?, ?it/s]

...

Epoch 28/30 | Train Loss: 824.4684 | Val Loss: 112.4822 | ROC AUC: 0.987 | PR AUC: 0.641 | MCC: 0.406


Epoch 29/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 29/30 | Train Loss: 825.4096 | Val Loss: 97.5090 | ROC AUC: 0.987 | PR AUC: 0.630 | MCC: 0.422


Epoch 30/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 30/30 | Train Loss: 628.7913 | Val Loss: 72.1604 | ROC AUC: 0.987 | PR AUC: 0.650 | MCC: 0.471


In [10]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
lr = 1e-3

model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=256, input_dim=input_dim,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

Training on: cpu (CPU)
[TensorBoard] Logging to: tensorboard_logs/baseline_h256_bs128_lr0.001_ep30_20250401-061858
[INFO] Loading Dataloader
[INFO] Initializing Streaming Dataset from DataFrame of size 2685
[INFO] Streaming V2 init done in 0.90 seconds
[INFO] Creating traindata with 3242943
[INFO] Creating valdata with 360327
Training model (hidden_dim=256) for 30 epochs...


/opt/anaconda3/envs/tf_keras/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 1/30 | Train Loss: 2053.8347 | Val Loss: 229.8138 | ROC AUC: 0.987 | PR AUC: 0.618 | MCC: 0.310


Epoch 2/30:   0%|          | 0/25336 [00:00<?, ?it/s]

...

Epoch 28/30 | Train Loss: 659.1683 | Val Loss: 92.5871 | ROC AUC: 0.986 | PR AUC: 0.621 | MCC: 0.428


Epoch 29/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 29/30 | Train Loss: 655.4341 | Val Loss: 64.2525 | ROC AUC: 0.987 | PR AUC: 0.639 | MCC: 0.486


Epoch 30/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 30/30 | Train Loss: 657.1748 | Val Loss: 107.0144 | ROC AUC: 0.988 | PR AUC: 0.652 | MCC: 0.422


In [11]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
lr = 1e-3

model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=512, input_dim=input_dim,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

Training on: cpu (CPU)
[TensorBoard] Logging to: tensorboard_logs/baseline_h512_bs128_lr0.001_ep30_20250401-072234
[INFO] Loading Dataloader
[INFO] Initializing Streaming Dataset from DataFrame of size 2685
[INFO] Streaming V2 init done in 0.90 seconds
[INFO] Creating traindata with 3242943
[INFO] Creating valdata with 360327
Training model (hidden_dim=512) for 30 epochs...


/opt/anaconda3/envs/tf_keras/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 1/30 | Train Loss: 1961.3288 | Val Loss: 159.6353 | ROC AUC: 0.989 | PR AUC: 0.622 | MCC: 0.363


Epoch 2/30:   0%|          | 0/25336 [00:00<?, ?it/s]

...

Epoch 28/30 | Train Loss: 643.1639 | Val Loss: 85.6609 | ROC AUC: 0.988 | PR AUC: 0.643 | MCC: 0.451


Epoch 29/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 29/30 | Train Loss: 647.4269 | Val Loss: 102.6664 | ROC AUC: 0.989 | PR AUC: 0.651 | MCC: 0.427


Epoch 30/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 30/30 | Train Loss: 644.3447 | Val Loss: 105.7595 | ROC AUC: 0.986 | PR AUC: 0.632 | MCC: 0.414


In [ ]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
lr = 1e-3

model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=1024, input_dim=input_dim,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

Training on: cpu (CPU)
[TensorBoard] Logging to: tensorboard_logs/baseline_h1024_bs128_lr0.001_ep30_20250401-085811
[INFO] Loading Dataloader
[INFO] Initializing Streaming Dataset from DataFrame of size 2685
[INFO] Streaming V2 init done in 0.90 seconds
[INFO] Creating traindata with 3242943
[INFO] Creating valdata with 360327
Training model (hidden_dim=1024) for 30 epochs...


/opt/anaconda3/envs/tf_keras/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 1/30 | Train Loss: 1957.2268 | Val Loss: 142.4008 | ROC AUC: 0.987 | PR AUC: 0.628 | MCC: 0.375


Epoch 2/30:   0%|          | 0/25336 [00:00<?, ?it/s]

...

Epoch 9/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 9/30 | Train Loss: 1528.3352 | Val Loss: 126.1394 | ROC AUC: 0.990 | PR AUC: 0.655 | MCC: 0.395


Epoch 10/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 10/30 | Train Loss: 1532.8266 | Val Loss: 124.6978 | ROC AUC: 0.990 | PR AUC: 0.663 | MCC: 0.408


Epoch 11/30:   0%|          | 0/25336 [00:00<?, ?it/s]

In [ ]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
lr = 1e-4

model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=1024, input_dim=input_dim,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

In [ ]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
lr = 1e-6

model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=1024, input_dim=input_dim,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

In [7]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
lr = 1e-5

model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=1024, input_dim=input_dim,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

Training on: cpu (CPU)
[TensorBoard] Logging to: tensorboard_logs/baseline_h1024_bs128_lr1e-05_ep30_20250401-022146
[INFO] Loading Dataloader
[INFO] Initializing Streaming Dataset from DataFrame of size 2685
[INFO] Streaming V2 init done in 0.60 seconds
[INFO] Creating traindata with 3242943
[INFO] Creating valdata with 360327
Training model (hidden_dim=1024) for 30 epochs...


/opt/anaconda3/envs/tf_keras/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 1/30 | Train Loss: 2746.4681 | Val Loss: 116.7348 | ROC AUC: 0.988 | PR AUC: 0.618 | MCC: 0.386


Epoch 2/30:   0%|          | 0/25336 [00:00<?, ?it/s]

...

Epoch 28/30 | Train Loss: 63.8310 | Val Loss: 40.9893 | ROC AUC: 0.986 | PR AUC: 0.640 | MCC: 0.559


Epoch 29/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 29/30 | Train Loss: 63.3009 | Val Loss: 42.6834 | ROC AUC: 0.986 | PR AUC: 0.631 | MCC: 0.549


Epoch 30/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 30/30 | Train Loss: 55.9526 | Val Loss: 39.0010 | ROC AUC: 0.986 | PR AUC: 0.638 | MCC: 0.565


In [9]:
#Load the best model
import torch
from model import ProteinClassifier
input_dim = cath_df.shape[1]-1
# Define your model architecture (must match the saved model)
model = ProteinClassifier(hidden_dim=1024, input_dim=input_dim)

# Load saved weights
model.load_state_dict(torch.load("./modelData/baseline_h1024_bs128_lr1e-05_ep30_20250401-022146_best.pt", map_location="cpu"))

# Set model to evaluation mode
model.eval()

/var/folders/vk/4clvfdvn29z3t_m4kf0z2y7c0000gn/T/ipykernel_42456/1880673432.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./modelData

ProteinClassifier(
  (linear1): Linear(in_features=3922, out_features=1024, bias=True)
  (batchnorm1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (linear2): Linear(in_features=1024, out_features=1, bias=True)
)

In [10]:
test_model_on_ecod(model, ecod_df)

[INFO] Initializing Streaming Dataset from DataFrame of size 761
[INFO] Streaming V2 init done in 0.34 seconds
[FINAL TEST on ECOD] Loss: 462.5683 | ROC AUC: 0.965 | PR AUC: 0.788 | MCC: 0.704


(462.5683311524644, 0.9652908960888762, 0.7884873464234206, 0.7042552401693547)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from dataset import StreamingProteinPairDataset, StreamingProteinPairDatasetV2
import time
import tracemalloc  # optional: for memory profiling

# Load a manageable subset for testing
df = pd.read_csv("data/cath_moments.tsv", sep='\t', header=None).dropna(axis=1).head(1000)

BATCH_SIZE = 64

def profile_loader(dataset_class, name=""):
    print(f"\n--- Profiling {name} ---")

    # Start memory profiling
    tracemalloc.start()
    start = time.time_ns()
    dataset = dataset_class(df)
    init_time_ns = time.time_ns() - start
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"[{name}] Init time: {init_time_ns / 1e6:.2f} ms")
    print(f"[{name}] Peak memory usage: {peak / (1024**2):.2f} MB")

    # Profile batch loading
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=4)
    batch_times_ns = []
    for i, (x, y) in enumerate(loader):
        if i >= 5:
            break
        t0 = time.time_ns()
        _ = x.shape, y.shape  # simulate some access
        t1 = time.time_ns()
        batch_times_ns.append(t1 - t0)

    if batch_times_ns:
        avg_batch_time_ms = sum(batch_times_ns) / len(batch_times_ns) 
        print(f"[{name}] Avg batch load time (first 5): {avg_batch_time_ms:.2f} ms")
    else:
        print(f"[{name}] No batches loaded.")

# Run profiling for both
profile_loader(StreamingProteinPairDataset, "Streaming v1")
profile_loader(StreamingProteinPairDatasetV2, "Streaming v2")